In [ ]:
# https://app.tavily.com/sign-in

In [ ]:
!pip install -U langchain langchain-classic langchain-community langchain-core

In [ ]:
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(k=5)
search.invoke("판교 근처 이탈리안 레스토랑은 무엇이 있나?")

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/소나기.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

split_docs = loader.load_and_split(text_splitter)
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [ ]:
docs = retriever.invoke(
    "소녀는 소년과 어디에서 처음 만났나?"
)
docs


In [ ]:
from langchain_core.tools import tool

@tool
def pdf_search(query: str) -> str:
    """PDF 문서로부터 정보를 검색한다. 소년과 소녀에 대한 내용은 이 도구를 사용한다."""
    docs = retriever.invoke(query)
    return "\n\n".join([d.page_content for d in docs])


In [ ]:
tools = [search, pdf_search]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)


In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="당신은 친절한 어시스턴트입니다. 사용자의 질문에 친절하게 답변하세요."
)


In [ ]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "2022년 월드컵 결승전 결과를 알려줘."}]
})

In [ ]:
print(response["messages"][-1].content)

In [ ]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "소년과 소녀는 어디에서 처음 만났나?"}]    
})
print(response["messages"][-1].content)


In [ ]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "안녕! 나는 판교에 살고 있는 개발자야. 만나서 반가와"}]    
})
print(response["messages"][-1].content)
